In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [ ]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

In [ ]:
# Keys to columns
# 4 fp flags (false positive flags)
# koi_period (orbital period in days)
# koi_time0bk (transit epoch)
# koi_impact (impact parameter)
# koi_duration (transit duration in hours)
# koi_depth (transit depth in ppm)
# koi_prad (planetary radius in Earth radii)
# koi_teq (equilibrium temperature in Kelvins)
# koi_insol (insolation flux in Earth flux)
# koi_model_snr (transit signal-to-noise)
# koi_tce_plnt_num (TCE planet number)
# koi_steff (stellar effective temperature in Kelvins)
# koi_slogg (stellar surface gravity logarithm)
# koi_srad (stellar radius)
# ra (RA - decimal degrees)
# dec (Dec - decimal degrees)
# koi_kepmag (Kepler-band in mag)

# Select your features (columns)

In [ ]:
# Set features. This will also be used as your x values.
selected_features = df[['names', 'of', 'selected', 'features', 'here']]

# Note: Do not select the error margins 
# Items of interest:
# koi_period (orbital period in days)
# koi_time0bk (transit epoch) - How does transit epoch differ from transit duration (below)? 
# koi_impact (impact parameter) - need more information.  What is this?  
# koi_duration (transit duration in hours)
# koi_depth (transit depth in ppm)
# koi_prad (planetary radius in Earth radii) - Hypothesis:  Smaller objects may be asteroids, not planets.
# koi_steff (stellar effective temperature in Kelvins)
# koi_slogg (stellar surface gravity logarithm) - Hypothesis:  Stronger surface gravity of stars may keep objects in orbit - indicator of planethood.
# koi_srad (stellar radius) - Hypothesis:  Smaller stars may be less likely to have planets. 

# Create a Train Test Split

Use `koi_disposition` for the y values

In [ ]:
y_values = df["koi_disposition"]
y_values.head()

In [ ]:
X_train.head()

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [ ]:
# Scale your data

# Train the Model



In [ ]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)